In [2]:
# generate submission codes to submit all parameter set simulations
import pandas as pd
import numpy as np
import os, glob, time, re, sys

In [6]:
iterflag = 0 # default
path_runmodel = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/SA_HH_allbasins/level1/run_model'
path_paramset = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/SA_HH_allbasins/level1/param_sets'
path_archive = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/SA_HH_allbasins/level1/ctsm_outputs'

script_singlerun = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/src/SA/run_one_paramset_SA.py'
script_clone = '/glade/u/home/guoqiang/CTSM_repos/CTSM_hillslope/cime/scripts/create_clone'
path_CTSM_base = '/glade/work/guoqiang/CTSM_CAMELS/SA_HH_allbasins/level1'

cases_per_submission = 10 # the number of model cases (i.e., parameter sets) per submission (<= 12 hours)

derecho_pbs = ['#PBS -N ctsmsa', 
               '#PBS -A P08010000',
               '#PBS -q main', 
               '#PBS -l job_priority=economy', 
               '#PBS -l walltime=12:00:00', 
               '#PBS -l select=3:ncpus=128', 
               '#PBS -e ./logs/',
               '#PBS -o ./logs/',
               'module load conda cdo nco', 'conda activate npl-2024a', 
               'export TMPDIR=/glade/derecho/scratch/$USER/temp', 
               'mkdir -p $TMPDIR']

os.makedirs(f'{path_runmodel}/logs', exist_ok=True)

In [7]:
# create command file
os.makedirs(path_runmodel, exist_ok=True)

pnum = 200
print('The number of total parameter sets:', pnum)

if np.mod(pnum, cases_per_submission)==0:
    subnum = int(pnum/cases_per_submission) 
else:
    subnum = int(pnum/cases_per_submission) + 1

for n in range(subnum):
    ns = int((n*cases_per_submission))
    ne = ns + cases_per_submission
    if ne > pnum:
        ne = pnum

    commands_run_model = f'{path_runmodel}/submit_iter{iterflag}_batch{n}.sh'

    with open(commands_run_model, 'w') as f:

        for l in derecho_pbs:
            _ = f.write(l + '\n')
        
        _ = f.write('\n')
        
        for i in range(ns, ne):
            caseflag = f'iter{iterflag}_trial{i}'
            file_parameter_set = f'{path_paramset}/paramset_iter{iterflag}_trial{i}.pkl'
            commandi = f"python {script_singlerun} {script_clone} {path_CTSM_base} {file_parameter_set} {path_archive} {caseflag}"

            _ = f.write('\n')
            _ = f.write(f"echo 'Processing parameter {file_parameter_set}'\n")
            _ = f.write(f'{commandi}\n')

        _ = f.write('\n')

The number of total parameter sets: 200
